# **Контест №2: Временные ряды**

**Выполнил:** Ибрагимов Роман Рифхатович

**Группа:** М8О-303Б-22

## **Постановка задачи**

Классическая задача прогноза продаж в сети магазинов.

Разные магазины в зависимости от различных факторов могут иметь различные продажи. Чтобы понять сколько товара нужно поставить в магазин, какая ожидается выручка и т.п. нам необходимо уметь предсказывать продажи в магазинах в зависимости от региона, сезона, температуры на улице, стоимости топлива, экономической ситуации и т.д.

Ваша задача предсказать продажи магазина на месяц вперед(4 недели) на основе истории.

Файлы
*   train.csv - тренировочный набор данных. С этим набором вы делаете все что хотите, там есть true значения и все фичи.
*   test.csv - тестовый набор данных. Здесь вам истинные значения будут не известны. Вы лишь можете сделать сабмит с помощью этого набора.

Описание столбцов
*   Store - ID магазина.
*   Date - дата на момент фиксации фич.
*   Weekly_Sales - продажи за неделю.
*   Temperature - средняя температура за неделю в городе расположения магазина(в градусах Фарингейта)
*   Fuel_Price - цена топлива в данном регионе.
*   CPI - индекс потребительских цен.
*   Unemployment - уровень безработици в регионе.

Ваша цель - предсказать Weekly_Sales на 4 измерения вперед (на месяц).

Порог прохождения лабораторки - MAPE(%) - 12% (Score on leaderboard < 0.12)

In [43]:
import sys
import warnings
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import mean_absolute_percentage_error

warnings.filterwarnings("ignore")

## **Блок загрузки данных**

Загрузим датасеты train.csv и test.csv.

In [44]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [45]:
train.head()

,Store,Date,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-01-10,1453329.50,71.89,2.603,211.671989,7.838
1,1,2010-02-04,1594968.28,62.27,2.719,210.820450,7.808
2,1,2010-02-07,1492418.14,80.91,2.669,211.223533,7.787
3,1,2010-02-19,1611968.17,39.93,2.514,211.289143,8.106
4,1,2010-02-26,1409727.59,46.63,2.561,211.319643,8.106


In [46]:
test.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment
0,1,2012-10-19,67.97,3.594,223.425723,6.573
1,1,2012-10-26,69.16,3.506,223.444251,6.573
2,1,2012-11-05,73.77,3.688,221.725663,7.143
3,1,2012-12-10,62.99,3.601,223.381296,6.573
4,2,2012-10-19,68.08,3.594,223.059808,6.170


In [47]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6255 non-null   int64  
 1   Date          6255 non-null   object 
 2   Weekly_Sales  6255 non-null   float64
 3   Temperature   6255 non-null   float64
 4   Fuel_Price    6255 non-null   float64
 5   CPI           6255 non-null   float64
 6   Unemployment  6255 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 342.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         180 non-null    int64  
 1   Date          180 non-null    object 
 2   Temperature   180 non-null    float64
 3   Fuel_Price    180 non-null    float64
 4   CPI           180 non-null    float64
 5   Unemployment  180 non-null  

## **Блок анализа данных**

Проанализируем временной ряд для 1-го магазина

In [48]:
store_data = train[train['Store'] == 1]
features = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

Построим сам временной ряд продаж.

In [49]:
time_series_fig = px.line(store_data, x=store_data.index, y='Weekly_Sales', title=f'Временной ряд продаж для магазина {1}')
time_series_fig.show()

А также сглаживание тренда с помощью скользящего среднего

In [50]:
store_data['Weekly_Sales_MA'] = store_data['Weekly_Sales'].rolling(window=4).mean()
smoothed_fig = px.line(store_data, x=store_data.index, y=['Weekly_Sales', 'Weekly_Sales_MA'],
                    title="Продажи и скользящее среднее")
smoothed_fig.show()

Произведём декомпозицию временного ряда на тренд, сезонность и отстатки.

In [51]:
decomposition = seasonal_decompose(store_data['Weekly_Sales'], model='additive', period=52)

trend_fig = px.line(x=store_data.index, y=decomposition.trend, title="Тренд временного ряда")
seasonal_fig = px.line(x=store_data.index, y=decomposition.seasonal, title="Сезонность временного ряда")
resid_fig = px.line(x=store_data.index, y=decomposition.resid, title="Остатки временного ряда")

trend_fig.show()
seasonal_fig.show()
resid_fig.show()

Для интереса посмотрим на аномалии (значения, которые сильно отклоняются от среднего)

In [53]:
store_data['Deviation'] = store_data['Weekly_Sales'] - store_data['Weekly_Sales_MA']
anomalies = store_data[store_data['Deviation'].abs() > 1.5 * store_data['Deviation'].std()]

anomaly_fig = px.line(store_data, x=store_data.index, y='Weekly_Sales', title="Аномалии во временном ряде продаж")
anomaly_fig.add_scatter(x=anomalies.index, y=anomalies['Weekly_Sales'], mode='markers',
                        marker=dict(color='red', size=10), name="Аномалии")
anomaly_fig.show()

Проверим ряд на стационарность.

In [54]:
adf_test = adfuller(store_data['Weekly_Sales'])
print("Тест Дики-Фуллера:")
print(f"ADF Статистика: {adf_test[0]}")
print(f"p-значение: {adf_test[1]}")
if adf_test[1] < 0.05:
    print("Ряд стационарен (p < 0.05).")
else:
    print("Ряд нестационарен (p >= 0.05).")

Тест Дики-Фуллера:
ADF Статистика: -9.446226845694193
p-значение: 4.747619877281164e-16
Ряд стационарен (p < 0.05).


На всякий случай посмотрим на уже дефолтные вещи, такие как распрделение признаков, попарные графики зависимости признаков и матрицу корреляций признаков.

In [55]:
def plot_histograms(data, feature):
    fig = px.histogram(data, x=feature, title=f"Распределение {feature} для магазина {1}")
    fig.show()

for feature in features:
        plot_histograms(store_data, feature)

In [56]:
pair_features = store_data[features]
pair_fig = px.scatter_matrix(pair_features, dimensions=features, title="Попарные графики признаков")
pair_fig.update_layout(height=900, width=900)
pair_fig.show()

In [57]:
corr_matrix = store_data[features].corr()

corr_fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale="RdBu",
    zmin=-1, zmax=1
))

corr_fig.update_layout(
    title='Матрица корреляций признаков для магазина',
    xaxis_nticks=36,
    width=600,
    height=600,
    margin=dict(l=100, r=100, t=100, b=100),
    coloraxis_colorbar=dict(
        title='Correlation'
    )
)

corr_fig.show()

Из последних двух графиков видно, что так-то наши признаки на таргет не оказывают влияния. По сути, их можно убрать из рассмотрения.

## **Блок подготовки и очистки данных**

Тут совсем мало, так как пропусков в датасетах нет, поэтому просто дату преобразуем в тип пандасовский тип даты.

In [58]:
train['Date'] = pd.to_datetime(train['Date'])
train.set_index('Date', inplace=True)
test['Date'] = pd.to_datetime(test['Date'])

## **Блок обучения и тестирования моделей**

Для получения предсказаний, воспользуюсь моделью ARIMA. Для каждого магазина своя отдельная модель. Сначала на валидационной выборке замерим метрику MAPE, затем обучим на всей выборке из датасета train и в конце сделаем предсказание на 1 месяц.

In [41]:
predictions = pd.DataFrame()

stores = train['Store'].unique()

forecast_steps = 4

all_actuals = []
all_forecasts = []

for store in stores:
    store_data = train[train['Store'] == store]['Weekly_Sales']

    train_data = store_data[:-forecast_steps]
    val_data = store_data[-forecast_steps:]

    try:
        model = ARIMA(train_data, order=(1, 1, 1))
        model_fit = model.fit()

        val_forecast = model_fit.forecast(steps=forecast_steps)

        mape = mean_absolute_percentage_error(val_data, val_forecast)
        print(f'MAPE для магазина {store}: {mape:.2%}')

        all_actuals.extend(val_data)
        all_forecasts.extend(val_forecast)

        model_full = ARIMA(store_data, order=(1, 1, 1))
        model_full_fit = model_full.fit()
        test_forecast = model_full_fit.forecast(steps=forecast_steps)

        last_date = store_data.index[-1]
        forecast_dates = pd.date_range(start=last_date + pd.Timedelta(weeks=1), periods=forecast_steps, freq='W')

        store_predictions = pd.DataFrame({
            'Store': store,
            'Date': forecast_dates,
            'Weekly_Sales': test_forecast
        })

        predictions = pd.concat([predictions, store_predictions], ignore_index=True)

    except Exception as e:
        print(f"Ошибка при обучении модели ARIMA для магазина {store}: {e}")


overall_mape = mean_absolute_percentage_error(all_actuals, all_forecasts)
print(f'\nОбщая MAPE для всех магазинов: {overall_mape:.2%}\n')

print(predictions.head())

MAPE для магазина 1: 6.64%
MAPE для магазина 2: 5.59%
MAPE для магазина 3: 6.48%
MAPE для магазина 4: 4.55%
MAPE для магазина 5: 6.04%
MAPE для магазина 6: 5.87%
MAPE для магазина 7: 10.42%
MAPE для магазина 8: 5.19%
MAPE для магазина 9: 5.71%
MAPE для магазина 10: 8.10%
MAPE для магазина 11: 6.39%
MAPE для магазина 12: 9.35%
MAPE для магазина 13: 2.54%
MAPE для магазина 14: 13.56%
MAPE для магазина 15: 6.95%
MAPE для магазина 16: 8.35%
MAPE для магазина 17: 6.76%
MAPE для магазина 18: 7.97%
MAPE для магазина 19: 4.53%
MAPE для магазина 20: 5.97%
MAPE для магазина 21: 7.57%
MAPE для магазина 22: 4.45%
MAPE для магазина 23: 2.63%
MAPE для магазина 24: 6.12%
MAPE для магазина 25: 3.33%
MAPE для магазина 26: 6.23%
MAPE для магазина 27: 7.16%
MAPE для магазина 28: 11.81%
MAPE для магазина 29: 5.19%
MAPE для магазина 30: 2.24%
MAPE для магазина 31: 4.46%
MAPE для магазина 32: 3.03%
MAPE для магазина 33: 8.80%
MAPE для магазина 34: 4.35%
MAPE для магазина 35: 4.45%
MAPE для магазина 36: 6.44

## **Блок сабмита**

Данные получены, сабмитим их.

In [42]:
data = {
    "Weekly_Sales": predictions['Weekly_Sales']
}
submit = pd.DataFrame(data)
submit.to_csv('submission.csv', index_label="ID")
print("Файл submission.csv готов к отправке")

Файл submission.csv готов к отправке
